<a href="https://colab.research.google.com/github/SamAdebisi/Stable_Diffusion/blob/main/DDPM_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Denoise Diffusion Probabilistic Image Generation with PyTorch

In [2]:
!pip3 install --upgrade pip
!pip3 install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
